<a href="https://colab.research.google.com/github/yardenfren1996/B-LoRA/blob/main/B_LoRA_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial

In [1]:
#0 kaggle; 1 google;2 rp 
runENV=0

if runENV==0:
    envOutPath='/kaggle/working'
    envInputPath='/kaggle/input'
if runENV==1:
    envOutPath='/content'
    envInputPath=envOutPath
if runENV==2:
    envOutPath='/workspace'
    envInputPath=envOutPath
    #for rp
    !pip install -q accelerate transformers wandb
    !apt-get update
    !apt-get install zip -y

In [2]:
!pip install -q diffusers==0.25.0 bitsandbytes
#for trainning (only need with latest version of diffusers)
# !pip uninstall peft -y

**Only run once**

In [ ]:
#only run the first time
!git clone https://github.com/RongWenYin/B-LoRA.git

In [3]:
%cd B-LoRA

/kaggle/working/B-LoRA


# Initial Params

In [4]:
styleKey='colorpen'
stylePath = f'img-{styleKey}'
# stylePath = "img-aut"
# stylePath = "s-autumn"
    
styleDir = f'{envInputPath}/{stylePath}'
# prompt_key = "[s90]"
prompt_key = "suliao"
layerKey='W45'
output_dir = f'outputs_{layerKey}_{stylePath}'
# output_dir = f'outputs_W25_{stylePath}'
# output_dir = f'outputs_w5_{stylePath}'
print(prompt_key)
print(styleDir)
print(output_dir)

suliao
/kaggle/input/img-colorpen
outputs_W45_img-colorpen


In [ ]:
import wandb
wandb.login()

# Training

In [15]:
#diy customer training-all blocks
!accelerate launch train_dreambooth_b-lora_sdxl.py \
 --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
 --instance_data_dir="{styleDir}" \
 --output_dir="{output_dir}" \
 --instance_prompt="{prompt_key}" \
 --resolution=1024 \
 --rank=64 \
 --train_batch_size=1 \
 --learning_rate=5e-5 \
 --lr_scheduler="constant" \
 --lr_warmup_steps=0 \
 --max_train_steps=1000 \
 --checkpointing_steps=1000 \
 --seed="0" \
 --gradient_checkpointing \
 --use_8bit_adam \
 --mixed_precision="fp16" \
 --cust_block_list="up_blocks.0.attentions.0 up_blocks.0.attentions.1"
#  --cust_block_list="down_blocks.2.attentions.1 up_blocks.0.attentions.1"
#  --cust_block_list="down_blocks.2.attentions.0 up_blocks.0.attentions.1"
#  --cust_block_list="up_blocks.0.attentions.1"
 # --report_to="wandb"  \
 # --cust_block_list="down_blocks.2.attentions.0 down_blocks.2.attentions.1 mid_block.attentions.0 up_blocks.0.attentions.0 up_blocks.0.attentions.1 up_blocks.0.attentions.2"
#  --cust_block_list="up_blocks.0.attentions.1"
 #  --cust_block_list="down_blocks.2.attentions.1 mid_block.attentions.0 up_blocks.0.attentions.0 up_blocks.0.attentions.1"

/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_p

# Inference Part : 

In [5]:
import torch
content_alpha,style_alpha = 1,1.1
def is_belong_to_blocks(key, blocks):
    try:
        for g in blocks:
            if g in key:
#                 print('add key..',key)
                return True
        return False
    except Exception as e:
        raise type(e)(f'failed to is_belong_to_block, due to: {e}')


def filter_lora(state_dict, blocks_):
    try:
        return {k: v for k, v in state_dict.items() if is_belong_to_blocks(k, blocks_)}
    except Exception as e:
        raise type(e)(f'failed to filter_lora, due to: {e}')


def scale_lora(state_dict, alpha):
    try:
        return {k: v * alpha for k, v in state_dict.items()}
    except Exception as e:
        raise type(e)(f'failed to scale_lora, due to: {e}')


def get_target_modules(unet, blocks=None):
    try:
        if not blocks:
            blocks = [('.').join(blk.split('.')[1:]) for blk in BLOCKS['content'] + BLOCKS['style']]

        attns = [attn_processor_name.rsplit('.', 1)[0] for attn_processor_name, _ in unet.attn_processors.items() if
                 is_belong_to_blocks(attn_processor_name, blocks)]

        target_modules = [f'{attn}.{mat}' for mat in ["to_k", "to_q", "to_v", "to_out.0"] for attn in attns]
        return target_modules
    except Exception as e:
        raise type(e)(f'failed to get_target_modules, due to: {e}')

#down_blocks.2.attentions.0 down_blocks.2.attentions.1 
#up_blocks.0.attentions.0 up_blocks.0.attentions.1 up_blocks.0.attentions.2
BLOCKS_M = {
    'content': ['unet.up_blocks.0.attentions.0'],
    'style': ['unet.up_blocks.0.attentions.1','unet.up_blocks.0.attentions.2'],
    'W1':['unet.down_blocks.2.attentions.0'],
    'W2':['unet.down_blocks.2.attentions.1'],
    'W3':['unet.mid_block.attentions.0'],
    'W4':['unet.up_blocks.0.attentions.0'],
    'W5':['unet.up_blocks.0.attentions.1'],
    'W6':['unet.up_blocks.0.attentions.2'],
    'W20':['down_blocks.2.attentions.1.transformer_blocks.0'],
    'W21':['down_blocks.2.attentions.1.transformer_blocks.1'],
    'W22':['down_blocks.2.attentions.1.transformer_blocks.2'],
    'W23':['down_blocks.2.attentions.1.transformer_blocks.3'],
    'W24':['down_blocks.2.attentions.1.transformer_blocks.4'],
    'W25':['down_blocks.2.attentions.1.transformer_blocks.5'],
    'W26':['down_blocks.2.attentions.1.transformer_blocks.6'],
    'W27':['down_blocks.2.attentions.1.transformer_blocks.7'],
    'W28':['down_blocks.2.attentions.1.transformer_blocks.8'],
    'W29':['down_blocks.2.attentions.1.transformer_blocks.9'],
    'W50':['unet.up_blocks.0.attentions.1.transformer_blocks.0'],
    'W51':['unet.up_blocks.0.attentions.1.transformer_blocks.1'],
    'W52':['unet.up_blocks.0.attentions.1.transformer_blocks.2'],
    'W53':['unet.up_blocks.0.attentions.1.transformer_blocks.3'],
    'W54':['unet.up_blocks.0.attentions.1.transformer_blocks.4'],
    'W55':['unet.up_blocks.0.attentions.1.transformer_blocks.5'],
    'W56':['unet.up_blocks.0.attentions.1.transformer_blocks.6'],
    'W57':['unet.up_blocks.0.attentions.1.transformer_blocks.7'],
    'W58':['unet.up_blocks.0.attentions.1.transformer_blocks.8'],
    'W59':['unet.up_blocks.0.attentions.1.transformer_blocks.9'],
#     'style': ['down_blocks.2.attentions.1','up_blocks.0.attentions.0','unet.up_blocks.0.attentions.1'],
}

# Function to free up GPU memory
def freeCache(pipeline):
    try:
        del pipeline  # Delete the pipeline object if it exists
        # print("Pipeline deleted to free up GPU memory.")
    except NameError:
        print("Pipeline does not exist, no need to delete.")
    except Exception as e:
        print(f"An error occurred when trying to delete pipeline: {e}")
    
    torch.cuda.empty_cache()  # Clear the GPU memory
    
# Optionally clear cache for transformers
import transformers
transformers.utils.move_cache()
        
def load_style_to_unet(pipe,layers, style_lora_model_id: str = '', content_alpha: float = 1.,
                            style_alpha: float = 1.) -> None:
        try:
            layerList=layers.split('_')
            blocks=[]
            for lay in layerList:
                blocks.extend(BLOCKS_M[lay])
#             print('blocks...')
#             print(blocks)
            # Get Style B-LoRA SD
            if style_lora_model_id:
                # print('loading lora from..',style_lora_model_id)
                style_B_LoRA_sd, _ = pipe.lora_state_dict(style_lora_model_id)
#                 print(style_B_LoRA_sd)
                #show all keyNames:
#                 print(list(style_B_LoRA_sd.keys()))
                print('use b-lorra..',blocks)
                style_B_LoRA = filter_lora(style_B_LoRA_sd, blocks)
                style_B_LoRA = scale_lora(style_B_LoRA, style_alpha)
            else:
                style_B_LoRA = {}

            # Merge B-LoRAs SD
#             res_lora = {**style_B_LoRA}
            
#             print('new keys..-')
#             print(style_B_LoRA)
#             for key in style_B_LoRA:
#                 print(key)

#             print('use b-lorra weight..-')
#             print(style_B_LoRA['unet.up_blocks.0.attentions.1.transformer_blocks.0.attn2.to_v.lora.up.weight'])
            # Load
            
            pipe.load_lora_into_unet(style_B_LoRA, None, pipe.unet)
        except Exception as e:
            raise type(e)(f'failed to load_b_lora_to_unet, due to: {e}')


import time
def genImagesBatch(layers, pipeline, objectNames,itemstep=1000):
    
    # Clear memory if necessary
    freeCache(pipeline)
#     time.sleep(.5)

    # Initialize the pipeline
    if pipeline is None:
        pipeline = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            vae=vae,
            torch_dtype=torch.float16,
        ).to("cuda")
        # print("Pipeline initialized.")

#         pipeline = StableDiffusionXLPipeline.from_pretrained(
#                     "stabilityai/stable-diffusion-xl-base-1.0",
#                     vae=vae,
#                     torch_dtype=torch.float16,
#                 )
#         pipeline = nn.DataParallel(pipeline)
#         pipeline = pipeline.to('cuda')
#         print("Pipeline initialized using DataParallel.")

    # Load style into the model
    load_style_to_unet(pipeline, layers, style_B_LoRA_path, content_alpha, style_alpha)
    
    print(f' {prompt_key} style | {layers} layer | {itemstep} step')

    # Generate images
    for objectName in objectNames:
        prompt = f'a {objectName} in {prompt_key} style | {layers} layer '
        # print(prompt)
        image = pipeline(prompt, generator=torch.Generator(device="cuda").manual_seed(138), num_images_per_prompt=1).images[0].resize((512, 512))
        image.save(f'{styleKey}__{objectName}__{layerKey}__{layers}__{itemstep}.png')
        torch.cuda.empty_cache()  # Free up GPU memory after saving each image

0it [00:00, ?it/s]

# GEN IMAGES IN BATCH

In [6]:
from diffusers import StableDiffusionXLPipeline, AutoencoderKL
from diffusers.utils import load_image, make_image_grid
from PIL import Image
import numpy as np
import torch
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16, use_safetensors=True)

/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import hf_hub_download
import os
# style_B_LoRA_path=hf_hub_download(repo_id="ElaineYin/fangao1", filename="f1_1000.safetensors", local_dir="lora100")
# style_B_LoRA_path=hf_hub_download(repo_id="ElaineYin/loras", filename="au_1000_W5.safetensors", local_dir="loras")
style_B_LoRA_path=hf_hub_download(repo_id="ElaineYin/loras", filename="suliao_original.safetensors", local_dir="loras")
print(style_B_LoRA_path)

**Generate image for multiple layers :**

In [8]:
# output_dir = f'outputs_{stylePath}'
output_dir = f'outputs_{layerKey}_{stylePath}'
print(output_dir)

outputs_W45_img-watercolor


In [22]:
objectNames = ["girl", "cat", "apple", "dog", "fish"]
# objectNames = ["girl", "cat", "fish"]
# objectNames = ["bike"]
# objectNames = ["",""]
layerList=['W4_W5','W5']
# layerList=['W2_W5']
# layerList=['W5']
# layerList = ['W1','W2','W3','W4','W5','W6']
# layerList = ['W2','W5','W1_W2','W1_W5','W2_W5','W3_W5','W4_W5','W5_W6']
# layerList = ['W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6']
# layerList = ['W1','W2','W3','W4','W5','W6','W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6']
# layerList = ['W1','W3','W4','W6','W1_W3','W1_W4','W1_W6','W2_W3','W2_W4','W2_W6','W3_W4','W3_W6','W4_W6']
#
# layerList = ['W1','W2','W3','W4','W5','W6','W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W6','W3_W6','W4_W6','W5_W6']

# layerList = ['W1_W2_W3','W1_W2_W4','W1_W2_W5','W1_W2_W6','W1_W3_W4','W1_W3_W5','W1_W3_W6','W1_W4_W5','W1_W4_W6','W1_W5_W6','W2_W3_W4','W2_W3_W5','W2_W3_W6','W2_W4_W5','W2_W4_W6','W2_W5_W6','W3_W4_W5','W3_W4_W6','W3_W5_W6','W4_W5_W6']
# layerList = ['W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6','W1_W2_W3','W1_W2_W4','W1_W2_W5','W1_W2_W6','W1_W3_W4','W1_W3_W5','W1_W3_W6','W1_W4_W5','W1_W4_W6','W1_W5_W6','W2_W3_W4','W2_W3_W5','W2_W3_W6','W2_W4_W5','W2_W4_W6','W2_W5_W6','W3_W4_W5','W3_W4_W6','W3_W5_W6','W4_W5_W6']
# layerList = ['W1','W2','W3','W4','W5','W6','W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6','W1_W2_W3','W1_W2_W4','W1_W2_W5','W1_W2_W6','W1_W3_W4','W1_W3_W5','W1_W3_W6','W1_W4_W5','W1_W4_W6','W1_W5_W6','W2_W3_W4','W2_W3_W5','W2_W3_W6','W2_W4_W5','W2_W4_W6','W2_W5_W6','W3_W4_W5','W3_W4_W6','W3_W5_W6','W4_W5_W6']

#W2345:
# layerList = ['W2','W3','W4','W5','W2_W3','W2_W4','W2_W5','W3_W4','W3_W5','W4_W5','W2_W3_W4','W2_W3_W5','W2_W4_W5','W3_W4_W5','W2_W3_W4_W5']
# layerList = ['W2_W3','W2_W4','W2_W5','W3_W4','W3_W5','W4_W5','W2_W3_W4','W2_W3_W5','W2_W4_W5','W3_W4_W5','W2_W3_W4_W5']
stepList=[1000]
# stepList=[1000,1500,2000]
pipeline = None  # Start with pipeline uninitialized
for itemstep in stepList:    
    style_B_LoRA_path = f'{output_dir}/checkpoint-{itemstep}/pytorch_lora_weights.safetensors'
#     style_B_LoRA_path = f'lora100/pytorch_lora_weights.safetensors'
    
    print('style_B_LoRA_path:',style_B_LoRA_path)
    # style_B_LoRA_path = f'{output_dir}/checkpoint-600/pytorch_lora_weights.safetensors'
    #from huggingface:
    # style_B_LoRA_path='ElaineYin/lora2345'
    # style_B_LoRA_path='/kaggle/input/lora_f2_alllayers_1200steps/pytorch/default/1/lora_f2_alllayers_1200steps.safetensors'
#     print(style_B_LoRA_path)
    for layers in layerList:
        genImagesBatch(layers, pipeline, objectNames,itemstep)

style_B_LoRA_path: outputs_W45_img-colorpen/checkpoint-1000/pytorch_lora_weights.safetensors


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

use b-lorra.. ['unet.up_blocks.0.attentions.0', 'unet.up_blocks.0.attentions.1']
 [s90] style | W4_W5 layer | 1000 step


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

**Generate image for multiple (SUB) layers :**

In [ ]:
output_dir = f'outputs_{stylePath}'
stepList=[1000]
objectNames = ["cat","dog","fish","girl"]
# objectNames = ["girl", "vase", "table", "tower", "fish"]
# layerList = ['W2_W5','W50','W51','W52','W53','W54','W55','W56','W57','W58','W59']
# layerList = ['W2','W5','W50_W51_W52_W53_W54_W55','W55_W56_W57_W58_W59']
# layerList = ['W50_W51_W52_W53_W54_W55_W56_W57_W58_W59','W51_W50_W52_W53_W54_W55_W56_W57_W58_W59','W55_W56_W57_W58_W59_W51_W50_W52_W53_W54','W51_W53_W57_W59_W55']

# layerList=[
# 'W50','W50_W51','W50_W51_W52','W50_W51_W52_W53','W50_W51_W52_W53_W54','W50_W51_W52_W53_W54_W55','W50_W51_W52_W53_W54_W55_W56','W50_W51_W52_W53_W54_W55_W56_W57','W50_W51_W52_W53_W54_W55_W56_W57_W58','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59',
# 'W51','W51_W52','W51_W52_W53','W51_W52_W53_W54','W51_W52_W53_W54_W55','W51_W52_W53_W54_W55_W56','W51_W52_W53_W54_W55_W56_W57','W51_W52_W53_W54_W55_W56_W57_W58','W51_W52_W53_W54_W55_W56_W57_W58_W59',
# 'W52','W52_W53','W52_W53_W54','W52_W53_W54_W55','W52_W53_W54_W55_W56','W52_W53_W54_W55_W56_W57','W52_W53_W54_W55_W56_W57_W58','W52_W53_W54_W55_W56_W57_W58_W59',
# 'W53','W53_W54','W53_W54_W55','W53_W54_W55_W56','W53_W54_W55_W56_W57','W53_W54_W55_W56_W57_W58','W53_W54_W55_W56_W57_W58_W59',
# 'W54','W54_W55','W54_W55_W56','W54_W55_W56_W57','W54_W55_W56_W57_W58','W54_W55_W56_W57_W58_W59',
# 'W55','W55_W56','W55_W56_W57','W55_W56_W57_W58','W55_W56_W57_W58_W59',
# 'W56','W56_W57','W56_W57_W58','W56_W57_W58_W59',
# 'W57','W57_W58','W57_W58_W59',
# 'W58','W58_W59'
# ]

# layerList=[
# 'W20','W20_W21','W20_W21_W22','W20_W21_W22_W23','W20_W21_W22_W23_W24','W20_W21_W22_W23_W24_W25','W20_W21_W22_W23_W24_W25_W26','W20_W21_W22_W23_W24_W25_W26_W27','W20_W21_W22_W23_W24_W25_W26_W27_W28','W20_W21_W22_W23_W24_W25_W26_W27_W28_W29',
# 'W21','W21_W22','W21_W22_W23','W21_W22_W23_W24','W21_W22_W23_W24_W25','W21_W22_W23_W24_W25_W26','W21_W22_W23_W24_W25_W26_W27','W21_W22_W23_W24_W25_W26_W27_W28','W21_W22_W23_W24_W25_W26_W27_W28_W29',
# 'W22','W22_W23','W22_W23_W24','W22_W23_W24_W25','W22_W23_W24_W25_W26','W22_W23_W24_W25_W26_W27','W22_W23_W24_W25_W26_W27_W28','W22_W23_W24_W25_W26_W27_W28_W29',
# 'W23','W23_W24','W23_W24_W25','W23_W24_W25_W26','W23_W24_W25_W26_W27','W23_W24_W25_W26_W27_W28','W23_W24_W25_W26_W27_W28_W29',
# 'W24','W24_W25','W24_W25_W26','W24_W25_W26_W27','W24_W25_W26_W27_W28','W24_W25_W26_W27_W28_W29',
# 'W25','W25_W26','W25_W26_W27','W25_W26_W27_W28','W25_W26_W27_W28_W29',
# 'W26','W26_W27','W26_W27_W28','W26_W27_W28_W29',
# 'W27','W27_W28','W27_W28_W29',
# 'W28','W28_W29'
# ]
# layerList=[
# 'W2_W50','W2_W50_W51','W2_W50_W51_W52','W2_W50_W51_W52_W53','W2_W50_W51_W52_W53_W54','W2_W50_W51_W52_W53_W54_W55','W2_W50_W51_W52_W53_W54_W55_W56','W2_W50_W51_W52_W53_W54_W55_W56_W57','W2_W50_W51_W52_W53_W54_W55_W56_W57_W58','W2_W50_W51_W52_W53_W54_W55_W56_W57_W58_W59',
# 'W2_W51','W2_W51_W52','W2_W51_W52_W53','W2_W51_W52_W53_W54','W2_W51_W52_W53_W54_W55','W2_W51_W52_W53_W54_W55_W56','W2_W51_W52_W53_W54_W55_W56_W57','W2_W51_W52_W53_W54_W55_W56_W57_W58','W2_W51_W52_W53_W54_W55_W56_W57_W58_W59',
# 'W2_W52','W2_W52_W53','W2_W52_W53_W54','W2_W52_W53_W54_W55','W2_W52_W53_W54_W55_W56','W2_W52_W53_W54_W55_W56_W57','W2_W52_W53_W54_W55_W56_W57_W58','W2_W52_W53_W54_W55_W56_W57_W58_W59',
# 'W2_W53','W2_W53_W54','W2_W53_W54_W55','W2_W53_W54_W55_W56','W2_W53_W54_W55_W56_W57','W2_W53_W54_W55_W56_W57_W58','W2_W53_W54_W55_W56_W57_W58_W59',
# 'W2_W54','W2_W54_W55','W2_W54_W55_W56','W2_W54_W55_W56_W57','W2_W54_W55_W56_W57_W58','W2_W54_W55_W56_W57_W58_W59',
# 'W2_W55','W2_W55_W56','W2_W55_W56_W57','W2_W55_W56_W57_W58','W2_W55_W56_W57_W58_W59',
# 'W2_W56','W2_W56_W57','W2_W56_W57_W58','W2_W56_W57_W58_W59',
# 'W2_W57','W2_W57_W58','W2_W57_W58_W59',
# 'W2_W58','W2_W58_W59'
# ]
# layerList = ['W2','W5']
pipeline = None  # Start with pipeline uninitialized
for itemstep in stepList:   
    style_B_LoRA_path = f'{output_dir}/checkpoint-{itemstep}/pytorch_lora_weights.safetensors'
    for layers in layerList:

        print('style_B_LoRA_path:',style_B_LoRA_path)
        genImagesBatch(layers, pipeline, objectNames,itemstep)

**Generate image for multiple layers :**

In [ ]:
objectNames = ["cat","dog"]
# objectNames = ["girl", "vase", "table", "tower", "fish"]
# layerList = ['W2_W5','W50','W51','W52','W53','W54','W55','W56','W57','W58','W59']
# layerList = ['W2','W5','W50_W51_W52_W53_W54_W55','W55_W56_W57_W58_W59']
layerList = ['W2_W3_W4_W5','W5_W3_W4_W2','W5_W4_W2_W3','W3_W2_W4_W5']

pipeline = None  # Start with pipeline uninitialized
for layers in layerList:
    genImagesBatch(layers, pipeline, objectNames)

# Normal Lora


In [7]:
pipeline = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            vae=vae,
            torch_dtype=torch.float16,
        ).to("cuda")

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


**load lora:**

In [8]:
# pipeline.load_lora_weights("ElaineYin/loras", weight_name="au_1000_W5.safetensors")
# pipeline.load_lora_weights("ElaineYin/loras", weight_name="suliao_original.safetensors")
# pipeline.load_lora_weights("ElaineYin/loras", weight_name="suliao_s90_layer5_1000.safetensors")
pipeline.load_lora_weights("ElaineYin/loras", weight_name="suliao_civitai.safetensors")


print('load lora done')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


suliao_civitai.safetensors:   0%|          | 0.00/228M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/diffusers/loaders/lora.py:805: FutureWarning: `_modify_text_encoder` is deprecated and will be removed in version 0.27. You are using an old version of LoRA backend. This will be deprecated in the next releases in favor of PEFT make sure to install the latest PEFT and transformers packages in the future.
  deprecate("_modify_text_encoder", "0.27", LORA_DEPRECATION_MESSAGE)
/opt/conda/lib/python3.10/site-packages/diffusers/loaders/lora.py:777: FutureWarning: `_remove_text_encoder_monkey_patch_classmethod` is deprecated and will be removed in version 0.27. You are using an old version of LoRA backend. This will be deprecated in the next releases in favor of PEFT make sure to install the latest PEFT and transformers packages in the future.
  deprecate("_remove_text_encoder_monkey_patch_classmethod", "0.27", LORA_DEPRECATION_MESSAGE)


load lora done


In [9]:
objectName="girl"
# styleKey='[s90]'
styleKey='suliao'
layers="W5"
itemstep="1000"
diyseed=1002
imgCnt=5
#todo no need add this,some pics use this as prompt:
# prompt = f'a {objectName} in {prompt_key} style | {layers} layer '
prompt = f'a {objectName} in {prompt_key} style'
print(f'{prompt}  | {layers} layer ')
for seedin in range(imgCnt):    
    image = pipeline(prompt, generator=torch.Generator(device="cuda").manual_seed(diyseed+seedin), num_images_per_prompt=1).images[0].resize((512, 512))
    image.save(f'{styleKey}__{objectName}__{layerKey}__{layers}__{itemstep}_{diyseed+seedin}.png')
    torch.cuda.empty_cache()  # Free up GPU memory after saving each image
# freeCache(pipeline)

a girl in suliao style  | W5 layer 


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

# **Zip and download images:**

In [12]:
!zip -j lora_civitai.zip {envOutPath}/B-LoRA/*.png
# !zip a_images.zip {envOutPath}/B-LoRA/*.png


  adding: suliao__girl__W45__W5__1000_1002.png (deflated 0%)
  adding: suliao__girl__W45__W5__1000_1003.png (deflated 0%)
  adding: suliao__girl__W45__W5__1000_1004.png (deflated 0%)
  adding: suliao__girl__W45__W5__1000_1005.png (deflated 0%)
  adding: suliao__girl__W45__W5__1000_1006.png (deflated 0%)


In [13]:
!rm {envOutPath}/B-LoRA/*.png

In [11]:
!rm {envOutPath}/B-LoRA/*.zip

**！！！Delete all files ---Run This carefully!!!:**

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
!rm {envOutPath}/B-LoRA/*.zip

# **Generate images with multiple styles:**

In [ ]:
from torch import nn
def genImagesBatchStyle(layers, pipeline, styleNames):
    
    # Clear memory if necessary
    freeCache(pipeline)
# #     time.sleep(1)

    # Initialize the pipeline
    if pipeline is None:
        pipeline = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            vae=vae,
            torch_dtype=torch.float16,
        ).to("cuda")
        print("Pipeline initialized.")

    # Load style into the model
    load_style_to_unet(pipeline, layers, style_B_LoRA_path, content_alpha, style_alpha)
#     time.sleep(1)

    # Generate images
    for styleName in styleNames:
#         prompt = f'a {prompt_key} in {styleName} style'
        prompt = f'a {styleName} of {prompt_key}'
        print(prompt)
        image = pipeline(prompt, generator=torch.Generator(device="cuda").manual_seed(138), num_images_per_prompt=1).images[0].resize((512, 512))
        image.save(f'{prompt_key}__{styleName}__{layers}.png')
        torch.cuda.empty_cache()  # Free up GPU memory after saving each image

    # Clean up and free memory after generating images
    freeCache(pipeline)

**Generate images with batch styles and layers list**

In [ ]:
styleNames = ["photorealistic portrait", "impressionist painting", "watercolor painting", "ukiyo-e style depiction", "oil painting","van gogh painting"]
# layerList=['W2','W3']
# layerList = ['W1','W2','W3','W4','W5','W6']
# layerList = ['W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6']
# layerList = ['W1_W2_W3','W1_W2_W4','W1_W2_W5','W1_W2_W6','W1_W3_W4','W1_W3_W5','W1_W3_W6','W1_W4_W5','W1_W4_W6','W1_W5_W6','W2_W3_W4','W2_W3_W5','W2_W3_W6','W2_W4_W5','W2_W4_W6','W2_W5_W6','W3_W4_W5','W3_W4_W6','W3_W5_W6','W4_W5_W6']
layerList = ['W1','W2','W3','W4','W5','W6','W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6','W1_W2_W3','W1_W2_W4','W1_W2_W5','W1_W2_W6','W1_W3_W4','W1_W3_W5','W1_W3_W6','W1_W4_W5','W1_W4_W6','W1_W5_W6','W2_W3_W4','W2_W3_W5','W2_W3_W6','W2_W4_W5','W2_W4_W6','W2_W5_W6','W3_W4_W5','W3_W4_W6','W3_W5_W6','W4_W5_W6']
pipeline = None  # Start with pipeline uninitialized
for layers in layerList:
    genImagesBatchStyle(layers, pipeline, styleNames)

# Inference with controlnet

**Only create Cany images once: **

In [ ]:
import cv2

def genCannyImg(imgurl):
    print('image url:',imgurl)
    original_image=Image.open(imgurl)
#     original_image = load_image(imgurl)

    image = np.array(original_image)

    low_threshold = 100
    high_threshold = 200

    image = cv2.Canny(image, low_threshold, high_threshold)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    canny_image = Image.fromarray(image)
    return canny_image

def genCannyImgList(sourcePath,imgList):
    for img in imgList:
        # canny_image=genCannyImg("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/sd_controlnet/hf-logo.png")
        canny_image=genCannyImg(f'{sourcePath}/{img}.jpg')
        canny_image.save(f'canny_{img}.png')
#         imageList.append(canny_image)
#     return imageList
sourcePath=f'/kaggle/input/canny-original'
objectNames = ["girl", "vase", "table", "tower", "fish"]
genCannyImgList(sourcePath,objectNames)

In [ ]:
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel, AutoencoderKL
from diffusers.utils import load_image, make_image_grid
from PIL import Image
import numpy as np
import torch
controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-canny-sdxl-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True
)
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16, use_safetensors=True)

# style_B_LoRA_path = f'/kaggle/input/color_building_w2345/pytorch/default/1/lora_color_building_W2345.safetensors'
style_B_LoRA_path = f'ElaineYin/color_building_W2345'
print(style_B_LoRA_path)

In [ ]:
# imgList=["https://plus.unsplash.com/premium_photo-1670282393309-70fd7f8eb1ef?fm=jpg&q=60&w=3000&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8Z2lybCUyMHBpY3R1cmV8ZW58MHx8MHx8fDA%3D",
#          "https://img.kwcdn.com/product/fancy/e4567ad7-634b-420b-9419-99bd4e56a5b2.jpg",
#          "https://img.zcdn.com.au/lf/8/hash/26242/19506358/4/Dirty%2BOak%2BFinish%2BShana%2BCoffee%2BTable.jpg",
#          "https://cdn.britannica.com/51/94351-050-86B70FE1/Leaning-Tower-of-Pisa-Italy.jpg",
#          "https://t4.ftcdn.net/jpg/02/74/20/69/360_F_274206901_Jt1PHZTbtwne17anw5eD9oABxStNJhYT.jpg"
#         ]
# objectNames = ["dog"]
objectNames = ["girl", "vase", "table", "tower", "fish"]
# sourcePath=f'/kaggle/input/canny-img'
sourcePath=f'/workspace'
def loadCannyImgList(sourcePath,objectNames):    
    imageList=[]
    for obj in objectNames:
        image=Image.open(f'{sourcePath}/canny_{obj}.png')
        imageList.append(image)
    return imageList
canny_images=loadCannyImgList(sourcePath,objectNames)

In [ ]:
import torch
import time
from diffusers import StableDiffusionXLPipeline, AutoencoderKL
def genControlImagesBatch(layers, pipeline, objectNames,canny_images,controlnet=None):
    
    # Clear memory if necessary
    freeCache(pipeline)
    # time.sleep(1)
    
    if controlnet:
        print('controlnet is not null')
    # Initialize the pipeline
    if pipeline is None:
       # Load the pipeline
        pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            vae=vae,
            controlnet=controlnet,
            torch_dtype=torch.float16,
            use_safetensors=True
        ).to("cuda")

        # Enable model CPU offload for memory efficiency
#         pipeline.enable_model_cpu_offload()

        # Move pipeline to CUDA
#         pipeline.to("cuda")

        # Optional: Enable attention slicing for memory efficiency during inference
#         pipeline.enable_attention_slicing()

        # Optional: Reduce memory footprint further if needed
#         pipeline.unet.enable_xformers_memory_efficient_attention()
        print("Pipeline initialized...o")
    load_style_to_unet(pipeline, layers, style_B_LoRA_path, content_alpha, style_alpha)
    # time.sleep(1)

    # Generate images
    for index, objectName in enumerate(objectNames):
        prompt = f'a {objectName} in {prompt_key} style'
        print(prompt)
        image = pipeline(prompt, 
                         generator=torch.Generator(device="cuda").manual_seed(138),
                         image=canny_images[index],
                         controlnet_conditioning_scale=1.0,
                         num_images_per_prompt=1).images[0].resize((512, 512))
        image.save(f'{prompt_key}__{objectName}__canny__{layers}.png')
        torch.cuda.empty_cache()  # Free up GPU memory after saving each image

    # Clean up and free memory after generating images
    freeCache(pipeline)

In [ ]:
layerList=['W2']
# layerList = ['W1','W2','W3','W4','W5','W6']
# layerList = ['W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6']
# layerList = ['W1_W2_W3','W1_W2_W4','W1_W2_W5','W1_W2_W6','W1_W3_W4','W1_W3_W5','W1_W3_W6','W1_W4_W5','W1_W4_W6','W1_W5_W6','W2_W3_W4','W2_W3_W5','W2_W3_W6','W2_W4_W5','W2_W4_W6','W2_W5_W6','W3_W4_W5','W3_W4_W6','W3_W5_W6','W4_W5_W6']
# layerList = ['W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6','W1_W2_W3','W1_W2_W4','W1_W2_W5','W1_W2_W6','W1_W3_W4','W1_W3_W5','W1_W3_W6','W1_W4_W5','W1_W4_W6','W1_W5_W6','W2_W3_W4','W2_W3_W5','W2_W3_W6','W2_W4_W5','W2_W4_W6','W2_W5_W6','W3_W4_W5','W3_W4_W6','W3_W5_W6','W4_W5_W6']
# layerList = ['W1','W2','W3','W4','W5','W6','W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6','W1_W2_W3','W1_W2_W4','W1_W2_W5','W1_W2_W6','W1_W3_W4','W1_W3_W5','W1_W3_W6','W1_W4_W5','W1_W4_W6','W1_W5_W6','W2_W3_W4','W2_W3_W5','W2_W3_W6','W2_W4_W5','W2_W4_W6','W2_W5_W6','W3_W4_W5','W3_W4_W6','W3_W5_W6','W4_W5_W6']

#W2345:
# layerList = ['W2','W3','W4','W5','W2_W3','W2_W4','W2_W5','W3_W4','W3_W5','W4_W5','W2_W3_W4','W2_W3_W5','W2_W4_W5','W3_W4_W5','W2_W3_W4_W5']
# layerList = ['W2_W3','W2_W4','W2_W5','W3_W4','W3_W5','W4_W5','W2_W3_W4','W2_W3_W5','W2_W4_W5','W3_W4_W5','W2_W3_W4_W5']
pipeline = None  # Start with pipeline uninitialized
for layers in layerList:
    genControlImagesBatch(layers, pipeline, objectNames,canny_images,controlnet=controlnet)